In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from itertools import permutations
import numpy as np
import scipy.io


In [ ]:
#load matlab files

Xd=scipy.io.loadmat(r'C:\Users\ansel\Dropbox (MIT)\Eye_tracking_Last\Eye_tracker_2_todo\4_Dataset_Healthy_vs_NeglectHemianopia\.mat')
y=scipy.io.loadmat(r'C:\Users\ansel\Dropbox (MIT)\Eye_tracking_Last\Eye_tracker_2_todo\4_Dataset_Healthy_vs_NeglectHemianopia\.mat')
ypat=scipy.io.loadmat(r'C:\Users\ansel\Dropbox (MIT)\Eye_tracking_Last\Eye_tracker_2_todo\4_Dataset_Healthy_vs_NeglectHemianopia\.mat')




X=Xd["X_all"]
y=y["Y_p"]
ypat=np.squeeze(ypat["ID_Tr"])
y=np.squeeze(y)

dimX=np.shape(X)
dimy=np.shape(y)

#randomply permute dataset and labels

perm = np.random.permutation(dimX[0])

X=np.take(X,perm,axis=0,out=X);
y=np.take(y,perm,axis=0,out=y);
ypat=np.take(ypat,perm,axis=0,out=ypat);


In [ ]:
#split train and test set

#split percentage
p=0.8
nt=int(np.floor(dimX[0]*p))

#train
Xtr=X[0:nt,:]
ytr=(y[0:nt]+1)/2

#test
Xts=X[nt+1:-1,:]
yts=(y[nt+1:-1]+1)/2

ytspat=ypat[nt+1:-1]


#get the dimensionality

dimXtr=np.shape(Xtr)
dimytr=np.shape(ytr)

dimXts=np.shape(Xts)
dimyts=np.shape(yts)



In [ ]:
# process the data to fit in a keras CNN properly
# input data needs to be (N, C, X, Y) - shaped where
# N - number of samples
# C - number of channels per sample
# (X, Y) - sample size

Xtr = Xtr.reshape((dimXtr[0], 1, dimXtr[1], 1))
Xts = Xts.reshape((dimXts[0], 1, dimXts[1], 1))

#categorical lebels
ytr = np_utils.to_categorical(ytr)
yts = np_utils.to_categorical(yts)


In [ ]:
# define a CNN

cnnbene = Sequential()

#2 conv layers
cnnbene.add(Conv2D(64, (3, 1),activation="relu",input_shape=(1, dimXtr[1], 1),padding="same"))
cnnbene.add(Conv2D(64, (3, 1), activation="relu",padding="same"))

#1max pooling layer
cnnbene.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_first"))

#3conv layers
cnnbene.add(Conv2D(128, (3, 1), activation="relu",padding="same"))
cnnbene.add(Conv2D(128, (3, 1), activation="relu",padding="same"))
cnnbene.add(Conv2D(128, (3, 1), activation="relu",padding="same"))

#1max pooling layer
cnnbene.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_first"))

#3 conv layers
cnnbene.add(Conv2D(256, (3, 1), activation="relu",padding="same"))
cnnbene.add(Conv2D(256, (3, 1), activation="relu",padding="same"))
cnnbene.add(Conv2D(256, (3, 1), activation="relu",padding="same"))

#1 max pooling layer
cnnbene.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_first"))

#2 fully connected layer with dropout regularization 
cnnbene.add(Flatten())
cnnbene.add(Dense(1024, activation="relu"))
cnnbene.add(Dropout(0.5))
cnnbene.add(Dense(2, activation="softmax"))



In [ ]:
# define optimizer and objective, compile cnn
cnnbene.compile(loss="categorical_crossentropy", optimizer='adagrad', metrics=['accuracy'])

In [ ]:
#sgd = SGD(lr=0.1, decay=1e-6, momentum=1, nesterov=True)
#cnnbene.compile(loss='categorical_crossentropy',
#              optimizer=sgd,
#              metrics=['accuracy'])

In [ ]:
# train
history=cnnbene.fit(Xtr, ytr, validation_data=(Xts,yts), epochs=15,verbose=2, validation_split=0.2)


In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
for i in range(12):
        indx=np.squeeze(np.argwhere(ytspat==i+1))
        score = cnnbene.evaluate(Xts[indx,:,:,:], yts[indx])
        print('Mean score', i+1,np.mean(score))

In [ ]:
for i in range(12):
    a=len(np.argwhere(ytspat==i+1))
    print(a)

In [ ]:
#from keras.models import load_model

#model.save('cnnbene')


#cnnbene.save('cnnft.h5')  # creates a HDF5 file 'my_model.h5'



In [ ]:
history.history['accuracy']

